In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, regexp_replace, trim, when
from pyspark.sql.types import IntegerType

In [2]:
spark = SparkSession.builder \
    .appName("AmazonReviewsSilver") \
    .config("spark.driver.memory", "6g") \
    .config("spark.executor.memory", "6g") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/16 01:07:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
#Carrega o Parquet
df = spark.read.format("parquet").load("/app/data/bronze/train_bronze")
df= df.toDF("label","title","review")


In [4]:
#Converter label para inteiro e tratar valores inválidos
df = df.withColumn("label",col("label").cast(IntegerType()))
df = df.filter(col("label").isin([1,2]))


In [5]:
#Tratar valores nulos: remover linhas que tiverem title ou review Nulos
df = df.filter(col("title").isNotNull() & col("review").isNotNull())
               

In [11]:
#Limpeza básica de texto: tirar quebras de linha, aspas duplicadas e espaços

df = df.withColumn("title", trim(regexp_replace(col("title"), r'\\n|"', '')))
df = df.withColumn("review", trim(regexp_replace(col("review"), r'\\n|"', '')))

df.show(10, truncate=False)


[Stage 5:>                                                          (0 + 1) / 1]

+-----+------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|title                                                 |review                                                                    

In [7]:
#Salva na camada Silver
df.write.mode("overwrite").format("parquet").save("/app/data/silver/train_silver")


In [8]:
spark.stop()
